#Problem 1

##Task 0

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('data.csv')
print(df.head())   # Print the first 5 columns of the dataset
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=42) # Splitting the data into train and test data


    Outlook  Temp Humidity Windy Play
0     Rainy   Hot     High     f   no
1     Rainy   Hot     High     t   no
2  Overcast   Hot     High     f  yes
3     Sunny  Mild     High     f  yes
4     Sunny  Cool   Normal     f  yes


##Task 1

In [2]:
def calculate_prior_probabilities(y_train):

    total_samples = len(y_train)           # Total number of samples in y_train
    yes_count = (y_train == 'yes').sum()   # Number of times yes occurs in y_train
    no_count = total_samples - yes_count   # Number of times no occurs in y_train
    prior_probabilities = {                # Formula to calculate the prior probabilities
        'yes': yes_count / total_samples,
        'no': no_count / total_samples
    }
    return prior_probabilities

# Calculate and print the prior probabilities
prior_probs = calculate_prior_probabilities(y_train)
print("Prior Probabilities:")
print("P(Play=yes):", prior_probs['yes'])
print("P(Play=no):", prior_probs['no'])

Prior Probabilities:
P(Play=yes): 0.5833333333333334
P(Play=no): 0.4166666666666667


##Task 2

In [3]:
def calculate_likelihood_probabilities(X_train, y_train):

    likelihoods = {}
    for feature in X_train.columns:
        likelihoods[feature] = {}
        for value in X_train[feature].unique():
            likelihoods[feature][value] = {}
            for outcome in ['yes', 'no']:
                # Calculate likelihood probability that is P(feature = value | Play = outcome)
                feature_outcome_count = len(X_train[(X_train[feature] == value) & (y_train == outcome)])
                outcome_count = len(y_train[y_train == outcome])
                likelihood = feature_outcome_count / outcome_count if outcome_count > 0 else 0  # if outcome_count = 0 , directly make the likelihood 0
                likelihoods[feature][value][outcome] = likelihood
    return likelihoods

# Calculate and print the likelihood probabilities
likelihood_probs = calculate_likelihood_probabilities(X_train, y_train)
print("\nLikelihood Probabilities:")
for feature, feature_probs in likelihood_probs.items():
    for value, outcome_probs in feature_probs.items():
        print(f"P({feature}={value}|Play=yes): {outcome_probs['yes']:.4f}")
        print(f"P({feature}={value}|Play=no): {outcome_probs['no']:.4f}")



Likelihood Probabilities:
P(Outlook=Rainy|Play=yes): 0.2857
P(Outlook=Rainy|Play=no): 0.6000
P(Outlook=Overcast|Play=yes): 0.4286
P(Outlook=Overcast|Play=no): 0.0000
P(Outlook=Sunny|Play=yes): 0.2857
P(Outlook=Sunny|Play=no): 0.4000
P(Temp=Hot|Play=yes): 0.2857
P(Temp=Hot|Play=no): 0.4000
P(Temp=Cool|Play=yes): 0.4286
P(Temp=Cool|Play=no): 0.2000
P(Temp=Mild|Play=yes): 0.2857
P(Temp=Mild|Play=no): 0.4000
P(Humidity=High|Play=yes): 0.2857
P(Humidity=High|Play=no): 0.8000
P(Humidity=Normal|Play=yes): 0.7143
P(Humidity=Normal|Play=no): 0.2000
P(Windy=f|Play=yes): 0.7143
P(Windy=f|Play=no): 0.4000
P(Windy=t|Play=yes): 0.2857
P(Windy=t|Play=no): 0.6000


##Task 3

In [4]:
def calculate_posterior_probabilities(X_test, prior_probs, likelihood_probs):

    posterior_probs = []
    for index, row in X_test.iterrows():
        probs = {}
        for outcome in ['yes', 'no']:
            prob = prior_probs[outcome]
            for feature, value in row.items():
                prob *= likelihood_probs[feature].get(value, {}).get(outcome, 0) # Handle missing values
            probs[outcome] = prob
        total_prob = sum(probs.values())
        normalized_probs = {outcome: prob / total_prob for outcome, prob in probs.items()}
        posterior_probs.append(normalized_probs)
    return posterior_probs

# Calculate posterior probabilities
posterior_probs = calculate_posterior_probabilities(X_test, prior_probs, likelihood_probs)


# Print the posterior probabilities
print("\nPosterior Probabilities:")
for i, probs in enumerate(posterior_probs):
    print(f"Instance {i+1}:")
    print(f"  P(Play=yes|Instance): {probs['yes']}")
    print(f"  P(Play=no|Instance): {probs['no']}")



Posterior Probabilities:
Instance 1:
  P(Play=yes|Instance): 0.8199947520335868
  P(Play=no|Instance): 0.18000524796641307
Instance 2:
  P(Play=yes|Instance): 1.0
  P(Play=no|Instance): 0.0


##Task 1

In [5]:
def predict_play(posterior_probs):

    predictions = []
    for probs in posterior_probs:
        if probs['yes'] > probs['no']:
            predictions.append('yes')
        else:
            predictions.append('no')
    return predictions

# Make predictions based on the posterior probabilities
predictions = predict_play(posterior_probs)

# Print our predictions
print("\nPredictions:")
for i, prediction in enumerate(predictions):
    # Also printing the actual y_test for checking
    actual_class = y_test.iloc[i]
    print(f"Instance {i+1}: Prediction - {prediction}, Actual - {actual_class}")




Predictions:
Instance 1: Prediction - yes, Actual - yes
Instance 2: Prediction - yes, Actual - yes


##Task 1

In [6]:
def calculate_likelihood_probabilities_laplace(X_train, y_train):

    likelihoods = {}
    for feature in X_train.columns:
        likelihoods[feature] = {}
        for value in X_train[feature].unique():
            likelihoods[feature][value] = {}
            for outcome in ['yes', 'no']:
                feature_outcome_count = len(X_train[(X_train[feature] == value) & (y_train == outcome)])
                outcome_count = len(y_train[y_train == outcome])
                # Laplace smoothing: add 1 to numerator and number of unique values to denominator
                likelihood = (feature_outcome_count + 1) / (outcome_count + len(X_train[feature].unique()))
                likelihoods[feature][value][outcome] = likelihood
    return likelihoods

# Calculate the likelihood probabilities with Laplace smoothing
likelihood_probs_laplace = calculate_likelihood_probabilities_laplace(X_train, y_train)

# Print the likelihood probabilities with Laplace smoothing
print("\nLikelihood Probabilities (Laplace Smoothed):")
for feature, feature_probs in likelihood_probs_laplace.items():
    for value, outcome_probs in feature_probs.items():
        print(f"P({feature}={value}|Play=yes): {outcome_probs['yes']:.4f}")
        print(f"P({feature}={value}|Play=no): {outcome_probs['no']:.4f}")

# Calculate the posterior probabilities with Laplace smoothing
posterior_probs_laplace = calculate_posterior_probabilities(X_test, prior_probs, likelihood_probs_laplace)

# Print the posterior probabilities with Laplace smoothing
print("\nPosterior Probabilities (Laplace Smoothed):")
for i, probs in enumerate(posterior_probs_laplace):
    print(f"Instance {i+1}:")
    print(f"  P(Play=yes|Instance): {probs['yes']}")
    print(f"  P(Play=no|Instance): {probs['no']}")

# Make predictions with Laplace smoothing
predictions_laplace = predict_play(posterior_probs_laplace)


# Compare both predictions
print("\nPredictions (Original):")
for i, prediction in enumerate(predictions):
    print(f"Instance {i+1}: {prediction}")

print("\nPredictions (Laplace Smoothed):")
for i, prediction in enumerate(predictions_laplace):
    print(f"Instance {i+1}: {prediction}")


Likelihood Probabilities (Laplace Smoothed):
P(Outlook=Rainy|Play=yes): 0.3000
P(Outlook=Rainy|Play=no): 0.5000
P(Outlook=Overcast|Play=yes): 0.4000
P(Outlook=Overcast|Play=no): 0.1250
P(Outlook=Sunny|Play=yes): 0.3000
P(Outlook=Sunny|Play=no): 0.3750
P(Temp=Hot|Play=yes): 0.3000
P(Temp=Hot|Play=no): 0.3750
P(Temp=Cool|Play=yes): 0.4000
P(Temp=Cool|Play=no): 0.2500
P(Temp=Mild|Play=yes): 0.3000
P(Temp=Mild|Play=no): 0.3750
P(Humidity=High|Play=yes): 0.3333
P(Humidity=High|Play=no): 0.7143
P(Humidity=Normal|Play=yes): 0.6667
P(Humidity=Normal|Play=no): 0.2857
P(Windy=f|Play=yes): 0.6667
P(Windy=f|Play=no): 0.4286
P(Windy=t|Play=yes): 0.3333
P(Windy=t|Play=no): 0.5714

Posterior Probabilities (Laplace Smoothed):
Instance 1:
  P(Play=yes|Instance): 0.7648247508884398
  P(Play=no|Instance): 0.23517524911156015
Instance 2:
  P(Play=yes|Instance): 0.49383604787186175
  P(Play=no|Instance): 0.5061639521281383

Predictions (Original):
Instance 1: yes
Instance 2: yes

Predictions (Laplace Smoo